# Results

Testing networks and saving results

TODO:
- clean up repeated bits into functions
- save csv files with results to share/use elsewhere

In [ ]:
# imports
import os
import numpy as np
import pandas as pd
import theano
import lasagne
import loading
from training import *
from network import *
from architectures import *
from scipy.stats import bayes_mvs, entropy, linregress, spearmanr

# aliases
L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor
bmvs = bayes_mvs

## Data Loading

In [ ]:
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/temp')
datadir = os.path.join(headdir, 'Data/model input')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')

data = loading.default_loader(os.path.join(datadir, '1-4 (no computer).csv'))
hvhdata = loading.default_loader(os.path.join(datadir, '0 (with groups).csv'))
df = hvhdata[0]
Xs = np.concatenate(hvhdata[2])
ys = np.concatenate(hvhdata[3])
Ss = np.concatenate(hvhdata[4])

## Compile Results

In [ ]:
num_filters = [4, 8, 16, 32, 64, 128]
archnames = ["arch{}".format(n) for n in num_filters]
columns = pd.MultiIndex.from_product([archnames, np.arange(5).astype(str)])
tune_tidy = pd.DataFrame(index=df.index, columns=columns)
tune_tidy['subject'] = df['subject']
tune_tidy['group'] = df['group'] - 1

pretrain_tidy = pd.DataFrame(index=df.index, columns=columns)
pretrain_tidy['subject'] = df['subject']
pretrain_tidy['group'] = df['group'] - 1

In [ ]:
for prefit_idx in range(5):
    for test_split in range(5):
        # create network container with nfil filters architecture
        fname = 'multiconvX_lrg {} agg fit exp 1-4 {} tune fit exp 0.npz'.format(prefit_idx, test_split)

        arch = multiconvX
        net = Network(arch)
        net.load_params(os.path.join(paramsdir, fname))

        group_idx = (test_split-1)%5 # fuck this up if you want to see overfitting
        test_data = df.loc[df['group']==(group_idx+1)]
        Xt, yt, St, Gt, Npt = loading.unpack_data(test_data)
        res = net.itemized_test_fn(Xt, yt)
        l1 = 'multiconvX'
        l2 = str(prefit_idx)
        tune_tidy.loc[tune_tidy['group']==group_idx, (l1, l2)] = res

    fname = 'multiconvX_lrg {} split agg fit exp 1-4.npz'.format(prefit_idx)

    arch = multiconvX
    net = Network(arch)
    net.load_params(os.path.join(paramsdir, fname))
    Xt, yt, St, Gt, Npt = loading.unpack_data(df)
    res = net.itemized_test_fn(Xt, yt)
    l1 = 'multiconvX'
    l2 = str(prefit_idx)
    pretrain_tidy[(l1, l2)] = res

In [ ]:
np.array([i.size for i in L.get_all_param_values(net.net)]).sum()

In [ ]:
pt = tune_tidy[['subject', 'group']].copy()
pt.columns = pt.columns.get_level_values(0)
pt['multiconvX'] = tune_tidy['multiconvX'].mean(axis=1)
ptpiv = pt.pivot_table(index='subject', values='multiconvX')

In [ ]:
bmvs(ptpiv.values)

In [ ]:
plt.hist(ptpiv.values, bins=np.arange(1.2, 2.8, .1))
sns.despine()

In [ ]:
for nfil in num_filters:
    for prefit_idx in range(5):
        for test_split in range(5):
            # create network container with nfil filters architecture
            if nfil == 32:
                fname = '{} agg fit exp 1-4 {} tune fit exp 0.npz'.format(prefit_idx, test_split)
            else:
                fname = 'arch{} {} agg fit exp 1-4 {} tune fit exp 0.npz'.format(nfil, prefit_idx, test_split)
            
            arch = lambda input_var: archX(input_var, num_filters=nfil)
            net = Network(arch)
            net.load_params(os.path.join(paramsdir, fname))
            
            group_idx = (test_split-1)%5 # fuck this up if you want to see overfitting
            test_data = df.loc[df['group']==(group_idx+1)]
            Xt, yt, St, Gt, Npt = loading.unpack_data(test_data)
            res = net.itemized_test_fn(Xt, yt)
            l1 = 'arch{}'.format(nfil)
            l2 = str(prefit_idx)
            tune_tidy.loc[tune_tidy['group']==group_idx, (l1, l2)] = res

        if nfil == 32:
            fname = '{} split agg fit exp 1-4.npz'.format(prefit_idx)
        else:
            fname = 'arch{} {} split agg fit exp 1-4.npz'.format(nfil, prefit_idx)
            
        arch = lambda input_var: archX(input_var, num_filters=nfil)
        net = Network(arch)
        net.load_params(os.path.join(paramsdir, fname))
        Xt, yt, St, Gt, Npt = loading.unpack_data(df)
        res = net.itemized_test_fn(Xt, yt)
        l1 = 'arch{}'.format(nfil)
        l2 = str(prefit_idx)
        pretrain_tidy[(l1, l2)] = res

In [ ]:
def pivtidy(tidydf):
    t = tidydf[archnames].astype(float)
    t = t.mean(axis=1, level=0)
    t['subject'] = tidydf['subject']
    t['group'] = tidydf['group']
    tpiv = t.pivot_table(index='subject', values=archnames)
    return tpiv

pretrain_piv = pivtidy(pretrain_tidy)
tune_piv = pivtidy(tune_tidy)

print('pretrain agg\n')
for arc in archnames:
    print(arc, "\n", bmvs(pretrain_tidy[arc].values), "\n")

print('\npretrain sub\n')
for arc in archnames:
    print(arc, "\n", bmvs(pretrain_piv[arc].values), "\n")

print('\ntune agg\n')
for arc in archnames:
    print(arc, "\n", bmvs(tune_tidy[arc].values), "\n")

print('\ntune sub\n')
for arc in archnames:
    print(arc, "\n", bmvs(tune_piv[arc].values), "\n")